In [1]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 3.3 MB/s eta 0:00:00a 0:00:01


In [2]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 55.0 MB/s eta 0:00:0000:0100:01


In [3]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 5.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 3.3 MB/s eta 0:00:00


In [4]:
# Ref:
# Faiss - Introduction to Similarity Search
# https://www.youtube.com/watch?v=sKyvsdEv6rk

# Colab Notebook
# https://colab.research.google.com/github/UKPLab/sentence-transformers/blob/master/examples/applications/retrieve_rerank/retrieve_rerank_simple_wikipedia.ipynb#scrollTo=UlArb7kqN3Re

In [5]:
import pandas as pd
import numpy as np
import os

In [6]:
base_path = '../input/movie-summaries-cmu/'

In [7]:
os.listdir('../input/movie-summaries-cmu')

['singapore.txt',
 'simplified_cmu_movie_data.csv',
 'singapore_rev1.txt',
 'MovieSummaries']

## Load the Singapore wikitravel file

In [8]:
file_path = base_path + 'singapore_rev1.txt'

# Open the file in 'r' mode (read mode)
with open(file_path, 'r') as file:
    # Read the entire contents of the file into a string
    file_contents = file.read()
    
# I added '###' to the file to be able to separate 
# the sections into chunks
#chunk_list = file_contents.split('###')
chunk_list = file_contents.split('\n\n')

# Remove newline (\n) characters
chunk_list= [item.strip() for item in chunk_list]

# Add "Singapore (Country)" to each list item so that
# each chunk will have context when the vectors are created
chunk_list = [f'Singapore (Country): {item}' for item in chunk_list]

#chunk_list[0].strip()

In [9]:
chunk_list[4]

'Singapore (Country): Singaporean food is legendary, with bustling hawker centres and 24-hour coffee shops offering cheap food from all parts of Asia, and shoppers can bust their baggage allowances in shopping centres like Orchard Road and Suntec City. In recent years some societal restrictions have also loosened up, and now you can bungee jump and dance on bar tops all night long, although alcohol is still very pricey and chewing gum can only be bought from a pharmacy for medical use.'

## Create the embedding vetors

In [10]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

# Sentences are encoded by calling model.encode()
embeddings = model.encode(chunk_list)

# Print the embeddings

"""
for chunk, embedding in zip(chunk_list, embeddings):
    print("Chunk:", chunk)
    print("Embedding:", embedding)
    print("")
"""

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

'\nfor chunk, embedding in zip(chunk_list, embeddings):\n    print("Chunk:", chunk)\n    print("Embedding:", embedding)\n    print("")\n'

In [11]:
embeddings.shape

(124, 384)

In [12]:
i = 1
print(chunk_list[i])
print(embeddings[i])

Singapore (Country): Singapore (Singapura, 新加坡) is a city-state in Southeast Asia that became independent from the British in 1965 and is now one of the world’s leading economic centre in shipping and banking. Combining the skyscrapers and subways of a modern, affluent city with a mix of Chinese, Malay, Arab, and Indian influences alongside a tropical climate, tasty food from hawker centres, copious shopping malls, and vibrant night-life scene, this Garden City makes a great stopover or springboard into the region.
[-1.77049392e-03  3.14408471e-03  1.45461485e-02  6.95298985e-02
  3.32952254e-02  2.12622769e-02  8.07667524e-02 -6.55521601e-02
 -8.40761960e-02  2.34035980e-02  7.35496730e-02 -1.31913930e-01
  6.11157436e-03  2.03160997e-02  2.82418188e-02 -1.83713436e-02
 -3.26464884e-02 -3.19764949e-02 -3.50251198e-02 -7.55463094e-02
 -5.45745008e-02 -6.23185784e-02 -3.20752077e-02  7.89341796e-03
  5.37340995e-03  2.68770326e-02  4.30875011e-02  2.04989798e-02
  2.15598615e-03  2.5630

## Set up FAISS - Exhaustive Search

In [13]:
import faiss

embed_length = embeddings.shape[1]

index = faiss.IndexFlatL2(embed_length)

In [14]:
index.is_trained

True

In [15]:
# Add the embeddings to the index
index.add(embeddings)

In [16]:
index.ntotal

124

In [17]:
query = ["How do I go to Johor Baru by taxi?"]
query_embedding = model.encode(query)

top_k = 3
D, I = index.search(query_embedding, top_k)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
I

array([[109, 110, 108]])

In [19]:
chunk_list[110]

'Singapore (Country): While normal taxis are not allowed to cross the border, specially licensed taxis can be taken from Larkin Bus Terminal in Johor Bahru (RM80 per taxi, or RM20 per person if you share with others), and Ban San St Taxi Kiosk (same place as Queen St Bus Terminal) in Singapore ($48 per taxi or $12 per person). Both Singapore-registered and Malaysian-registered taxis are available. Singapore-registered taxis can bring you to anywhere in Singapore but can only go to Larkin in Johor Bahru, while Malaysian-registered taxis can bring you to anywhere in Malaysia but can only go to Ban San St in Singapore. Drop-off points other than the taxi terminal in the destination country may incur additional charges; check with the driver before boarding. Booking is available by phone from Pengurusan Terminal Teksi Johor Bahru-Singapore (Malaysian taxis) at ☎ +60 7 222 5898 or +60 7 224 6986, and Singapore-Johore Taxi Operators’ Association (Singapore taxis) at ☎ +65 6296 7054.'

## Set up FAISS - Approximate Search

In [20]:
# How many clusters (voronoid cells) do we want?
# Example: For 4 centroilds we need at least 156 embeddings in
# order to train the index.
num_centroids = 2

quantizer = faiss.IndexFlatL2(embed_length)

index = faiss.IndexIVFFlat(quantizer, embed_length, num_centroids)

index.is_trained

False

In [21]:
# Train the index
# After the index is trained it's ready to receive data

index.train(embeddings)

index.is_trained

True

In [22]:
# Add the embeddings to the index

index.add(embeddings)

index.ntotal

124

In [23]:
query = ["How do I go to Johor Baru by taxi?"]
query_embedding = model.encode(query)

top_k = 3
D, I = index.search(query_embedding, top_k)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
I

array([[109, 110, 108]])

In [25]:
chunk_list[109]

'Singapore (Country): By taxi\nThere is also the option of taking a taxi between Singapore and Johor Bahru. The main advantage is that you do not need to lug your stuff (or yourself) through Immigration and Customs at both ends; you can just sit in the car.'

In [26]:
# So far we've just be searching the cell with 
# the nearest centroid.
# Setting nprobe allows us to search more of
# the nearest cells. e.g. nprobe = 4 means w will search 4 cells.
# This can be done if we were not getting good results and wanted
# to improve performance. The time taken also increases as we are
# comparing to more vectors.

index.nprobe = 4

In [27]:
query = ["How do I go to Johor Baru by taxi?"]
query_embedding = model.encode(query)

top_k = 3
D, I = index.search(query_embedding, top_k)

I

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

array([[109, 110, 108]])

In [28]:
# There's also one other method to spped up the search.
# Please refer to the video.

## Re-ranking the predicted results

In [29]:
from sentence_transformers import CrossEncoder

# We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [30]:
query = ["How do I go to Johor Baru by taxi?"]
query_embedding = model.encode(query)

top_k = 10
D, I = index.search(query_embedding, top_k)

list(I[0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[109, 110, 108, 99, 85, 100, 96, 98, 89, 111]

In [31]:
pred_list = list(I[0])

# Replace the chunk index values with the corresponding strings
pred_strings_list = [chunk_list[item] for item in pred_list]

pred_strings_list[0]

'Singapore (Country): By taxi\nThere is also the option of taking a taxi between Singapore and Johor Bahru. The main advantage is that you do not need to lug your stuff (or yourself) through Immigration and Customs at both ends; you can just sit in the car.'

In [32]:
# This input to the cross_encoder is a list of lists
# [[query_text, pred_text1], [query_text, pred_text2], ...]

cross_input_list = []

for item in pred_strings_list:
    
    new_list = [query[0], item]
    
    cross_input_list.append(new_list)
    

In [33]:
cross_input_list[2]

['How do I go to Johor Baru by taxi?',
 'Singapore (Country): Bus to/from MRT: Buses connect the train checkpoint with Kranji, Marsiling and Woodlands MRT stations. The nearest station is Marsiling (1.6 km away), but more buses go to Woodlands. From MRT stations, ensure that the bus goes to Woodlands Train Checkpoint; ask the driver to be certain that the bus goes to the "KTM station".\nBus to/from city: At the train checkpoint bus stop, SBS 170 (red plate) goes via Kranji MRT station to Queen St in the city, as well as Larkin Sentral in Johor Bahru in the other direction. Pay attention to the bus destination sign or ask the driver to ensure you board the correct bus.\nAlternatively, use the overhead bridge (following signs to "Woodlands Checkpoint") to cross over to the bus arrival hall at the road checkpoint and take an express bus to the city. This option is less useful as you have to wait for the express bus to come from across the border, and you have to pay the fare in Singapore 

In [34]:
# Put the pred text into a dataframe

df = pd.DataFrame(cross_input_list, columns=['query_text', 'pred_text'])
df['original_index'] = I[0]

df.head()

,query_text,pred_text,original_index
0,How do I go to Johor Baru by taxi?,Singapore (Country): By taxi\nThere is also th...,109
1,How do I go to Johor Baru by taxi?,Singapore (Country): While normal taxis are no...,110
2,How do I go to Johor Baru by taxi?,Singapore (Country): Bus to/from MRT: Buses co...,108
3,How do I go to Johor Baru by taxi?,Singapore (Country): The most popular options ...,99
4,How do I go to Johor Baru by taxi?,Singapore (Country): From the airport you woul...,85


In [35]:
# Now, score all retrieved passages with the cross_encoder
cross_scores = cross_encoder.predict(cross_input_list)

cross_scores

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

array([ 6.562726  ,  4.802743  , -0.4356162 ,  2.7365267 , -5.670985  ,
       -0.4665946 , -8.342102  ,  1.4866202 ,  0.44983345, -4.666091  ],
      dtype=float32)

In [36]:
# Add the scores to the dataframe

df['cross_scores'] = cross_scores

df.head()

,query_text,pred_text,original_index,cross_scores
0,How do I go to Johor Baru by taxi?,Singapore (Country): By taxi\nThere is also th...,109,6.562726
1,How do I go to Johor Baru by taxi?,Singapore (Country): While normal taxis are no...,110,4.802743
2,How do I go to Johor Baru by taxi?,Singapore (Country): Bus to/from MRT: Buses co...,108,-0.435616
3,How do I go to Johor Baru by taxi?,Singapore (Country): The most popular options ...,99,2.736527
4,How do I go to Johor Baru by taxi?,Singapore (Country): From the airport you woul...,85,-5.670985


In [37]:
# Sort the DataFrame in descending order based on the scores

df_sorted = df.sort_values(by='cross_scores', ascending=False)

df_sorted.head(10)

,query_text,pred_text,original_index,cross_scores
0,How do I go to Johor Baru by taxi?,Singapore (Country): By taxi\nThere is also th...,109,6.562726
1,How do I go to Johor Baru by taxi?,Singapore (Country): While normal taxis are no...,110,4.802743
3,How do I go to Johor Baru by taxi?,Singapore (Country): The most popular options ...,99,2.736527
7,How do I go to Johor Baru by taxi?,Singapore (Country): An alternative to taking ...,98,1.486620
8,How do I go to Johor Baru by taxi?,Singapore (Country): The Causeway is a very po...,89,0.449833
2,How do I go to Johor Baru by taxi?,Singapore (Country): Bus to/from MRT: Buses co...,108,-0.435616
5,How do I go to Johor Baru by taxi?,Singapore (Country): By train\nMalaysia's Kere...,100,-0.466595
9,How do I go to Johor Baru by taxi?,Singapore (Country): A combination ride from a...,111,-4.666091
4,How do I go to Johor Baru by taxi?,Singapore (Country): From the airport you woul...,85,-5.670985
6,How do I go to Johor Baru by taxi?,"Singapore (Country): KKKL, ☎ +65 6348 6909, [x...",96,-8.342102


In [38]:
# Compare the orginal order and the re-ranked order

print(I[0])
print(list(df_sorted['original_index']))

[109 110 108  99  85 100  96  98  89 111]
[109, 110, 99, 98, 89, 108, 100, 111, 85, 96]


In [39]:
for i in range(0,3):
    
    text = df_sorted.loc[i, 'pred_text']
    print(text)
    print()

Singapore (Country): By taxi
There is also the option of taking a taxi between Singapore and Johor Bahru. The main advantage is that you do not need to lug your stuff (or yourself) through Immigration and Customs at both ends; you can just sit in the car.

Singapore (Country): While normal taxis are not allowed to cross the border, specially licensed taxis can be taken from Larkin Bus Terminal in Johor Bahru (RM80 per taxi, or RM20 per person if you share with others), and Ban San St Taxi Kiosk (same place as Queen St Bus Terminal) in Singapore ($48 per taxi or $12 per person). Both Singapore-registered and Malaysian-registered taxis are available. Singapore-registered taxis can bring you to anywhere in Singapore but can only go to Larkin in Johor Bahru, while Malaysian-registered taxis can bring you to anywhere in Malaysia but can only go to Ban San St in Singapore. Drop-off points other than the taxi terminal in the destination country may incur additional charges; check with the dri

## Use OpenAI to create a natural language output

In [40]:
question = 'How do I go to Johor Baru by taxi?'
pred_text_list = list(df_sorted['pred_text'])
context = pred_text_list[0:3]

# Prepare the prompt
prompt = f"""
Excerpt from the Wikitravel page for Singapore: 
{context}
Question: {question}

Extract the answer of the question from the text provided. 
If the text doesn't contain the answer, 
reply that the answer is not available."""

In [41]:
from openai import OpenAI

api_key  = 'Your-API-Key'

client = OpenAI(api_key=api_key)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo-0301",
  messages=[
    {"role": "system", "content": "You are a helpful travel assistant."},
    {"role": "user", "content": prompt}
  ]
)


print(completion.choices[0].message.content)

To go to Johor Bahru by taxi, you can take specially licensed taxis from Larkin Bus Terminal in Johor Bahru or Ban San St Taxi Kiosk in Singapore. Singapore-registered taxis can bring you to anywhere in Singapore but can only go to Larkin in Johor Bahru, while Malaysian-registered taxis can bring you to anywhere in Malaysia but can only go to Ban San St in Singapore. Drop-off points other than the taxi terminal in the destination country may incur additional charges; check with the driver before boarding. Booking is available by phone from Pengurusan Terminal Teksi Johor Bahru-Singapore (Malaysian taxis) or Singapore-Johore Taxi Operators' Association (Singapore taxis).


## Summary of the RAG process

[ 1 ] Setup
1. Read in the text from the file
2. Chunk the text and store the chunks in a list
3. Use Sentence Transformer to create a vector for each chunk
4. Use FAISS to create the index (Set up either greedy search or nearest neighbor search)
5. Train the index, if needed
6. Load the vectors into the index
7. Set up re-rank

[2] Run similarity search
1. Get the query text (question)
2. Vector encode the question
3. Run similarity search

[3] Run re-rank

[4] Run OpenAi
1. Get the question
2. Get the context
2. Create the prompt
3. Run the prompt through OpenAi
4. Get the natural language answer to the question